<a href="https://colab.research.google.com/github/MCygert/preparing_for_tensorflow_certificate/blob/main/08_intro_to_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP fundamentals

In [1]:
!nvidia-smi

Sun Jan 23 13:22:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-01-23 13:22:53--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-01-23 13:22:53 (103 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [73]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a text dataset

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data("nlp_getting_started.zip")

--2022-01-23 13:22:56--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 173.194.202.128, 74.125.199.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-01-23 13:22:56 (148 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

In [5]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [6]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [7]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [9]:
# Let's visualize random data
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")


Target: 1 (real disaster)
Text:
Hi-Res Doppler showing storm just NE of Edmond is now severe with 1.25' hail and 60mph wind possible. #okwx http://t.co/lrx0sDsNHM

---

Target: 1 (real disaster)
Text:
@godsfirstson1 and she wrapped his coat around herself. It practically engulfed her.

---

Target: 0 (not real disaster)
Text:
theres a fire truck in this parking lot

---

Target: 0 (not real disaster)
Text:
Turn back to me! I have rescued you and swept away your sins as though they were clouds. Isaiah 44:22 Contemporary English Version

---

Target: 0 (not real disaster)
Text:
@amaramin3 Meek is definitely capable of body bagging his ass on the track Drake was just smooth as fuck with it!

---



## Split data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
# Use train_test_split to split training data into trainnig and validaiton sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [12]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [13]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

In [14]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [15]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
max_vocab_length = 10000
max_length = 15 
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)


In [17]:
text_vectorizer.adapt(train_sentences)

In [18]:
sample_sentence = " There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_words = words_in_vocab[:5]
bottom_words = words_in_vocab[-5:]
top_words, bottom_words

(['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

### Creating an Embedding using Tensorflow layer

In [20]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 
embedding

In [21]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \n\nEmbeded version:")
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 This is a tragedy: I added the wrong book to my TBR list now I can't find the right one. This is what comes of browsing just on gr homepage. 

Embeded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 1.6911063e-02, -3.5824738e-02,  4.9727011e-02, ...,
          2.0761494e-02,  2.6369940e-02,  4.1201677e-02],
        [ 1.8037446e-03,  2.9466163e-02, -3.1711448e-02, ...,
          1.1349760e-02,  1.0585390e-02,  4.9907062e-02],
        [-2.2559179e-02, -5.9377439e-03,  2.1177698e-02, ...,
          3.9062154e-02, -2.7693629e-02,  2.2688556e-02],
        ...,
        [ 1.9629631e-02,  2.7381588e-02, -1.2098443e-02, ...,
         -6.0714781e-05, -1.0007143e-02,  5.8340058e-03],
        [ 1.6850855e-02, -2.0074248e-03,  3.1791653e-02, ...,
         -1.5110325e-02,  1.4847580e-02,  2.0385388e-02],
        [ 9.2537291e-03, -2.4999773e-02, -3.1745721e-02, ...,
          2.7062956e-02, -4.4059563e-02,  4.4357430e-02]]], dtype=float32)>

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])

# Fit the pipeline
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [23]:
# Evaluate our baseline model
baseline_scores = model_0.score(val_sentences, val_labels)
baseline_scores

0.7926509186351706

In [24]:
baseline_preds  = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [25]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
  model_results = {'accuracy': model_accuracy,
                   'precision': model_precision,
                   'recall': model_recall,
                   'f1': model_f1}
  return model_results


In [26]:
# Get baseline results
baseline_results = calculate_results(val_labels, baseline_preds)

In [27]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

## Feed forward model

In [28]:
# Create a tensorboard callback
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard lgos
SAVE_DIR = 'model_logs'


In [29]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [30]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [31]:
# Compile model
model_1.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [32]:
#Fit the model
model_1_history = model_1.fit(train_sentences,
                                  train_labels,
                                  epochs=5,
                                  validation_data=(val_sentences, val_labels),
                                  callbacks=[create_tensorboard_callback(dir_name='model_logs', experiment_name='model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20220123-132300
Epoch 1/5
215/215 [==============================] - 3s 5ms/step - loss: 0.6117 - accuracy: 0.6875 - val_loss: 0.5363 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4423 - accuracy: 0.8168 - val_loss: 0.4696 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3466 - accuracy: 0.8589 - val_loss: 0.4584 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2835 - accuracy: 0.8897 - val_loss: 0.4646 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2374 - accuracy: 0.9127 - val_loss: 0.4873 - val_accuracy: 0.7927


In [33]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4873 - accuracy: 0.7927


[0.4872627556324005, 0.7926509380340576]

In [34]:
model_1_pred_probs=model_1.predict(val_sentences)
model_1_pred_probs.shape

(762, 1)

In [35]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [36]:
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 79.26509186351706,
 'f1': 0.7889824348879466,
 'precision': 0.8002882489930304,
 'recall': 0.7926509186351706}

In [37]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [38]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False,  True])

In [39]:
# Get the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:15]

(10000,
 ['',
  '[UNK]',
  'the',
  'a',
  'in',
  'to',
  'of',
  'and',
  'i',
  'is',
  'for',
  'on',
  'you',
  'my',
  'with'])

In [40]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [41]:
embed_weights = model_1.get_layer('embedding_1').get_weights()[0]
embed_weights, embed_weights.shape

(array([[ 0.01759782, -0.01750564, -0.03577095, ..., -0.05790439,
          0.01734677,  0.00640563],
        [-0.01437184,  0.03414277, -0.04164734, ..., -0.01413277,
          0.04903411,  0.02679081],
        [ 0.05725219,  0.00537669, -0.05655055, ..., -0.00117415,
         -0.03289582,  0.01496135],
        ...,
        [-0.0341761 , -0.00773945, -0.02922107, ...,  0.02229451,
          0.0366346 ,  0.04561836],
        [ 0.00199139,  0.064349  , -0.03272744, ..., -0.01195653,
          0.02809778, -0.05547362],
        [ 0.03999592,  0.06858867, -0.03902323, ..., -0.05678393,
          0.06801884, -0.02330013]], dtype=float32), (10000, 128))

In [42]:
 import io

# Create output writers
out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# Write embedding vectors and words to file
for num, word in enumerate(words_in_vocab):
  if num == 0: 
     continue # skip padding token
  vec = embed_weights[num]
  out_m.write(word + "\n") # write words to file
  out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
out_v.close()
out_m.close()
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download("embedding_vectors.tsv")
  files.download("embedding_metadata.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Network (RNN)

In [43]:
# Create an LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x)
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_2 = tf.keras.Model(inputs,outputs, name='model_2_LSTM')

In [44]:
# Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [45]:
model_2.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])


In [46]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 'model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220123-132313
Epoch 1/5
215/215 [==============================] - 5s 8ms/step - loss: 0.2318 - accuracy: 0.9140 - val_loss: 0.5122 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1588 - accuracy: 0.9393 - val_loss: 0.5987 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1281 - accuracy: 0.9515 - val_loss: 0.7275 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1052 - accuracy: 0.9593 - val_loss: 0.7643 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0871 - accuracy: 0.9641 - val_loss: 0.9588 - val_accuracy: 0.7769


In [47]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[1.7382145e-02],
       [8.9085919e-01],
       [9.9960881e-01],
       [2.9355848e-02],
       [2.4311809e-04],
       [9.9544322e-01],
       [8.0280298e-01],
       [9.9976963e-01],
       [9.9962378e-01],
       [4.8600394e-01]], dtype=float32)

In [48]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [49]:
model_2_results = calculate_results(val_labels,
                                    model_2_preds)
model_2_results

{'accuracy': 77.69028871391076,
 'f1': 0.774621521428632,
 'precision': 0.779247157719032,
 'recall': 0.7769028871391076}

In [50]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [51]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# x = layers.LSTM(42, return_sequences=True)(x)
# x = layers.GRU(99)(x)
# x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name='model_3_GRU')


In [52]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [53]:
model_3.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [54]:
model_3_history = model_3.fit(train_sentences, train_labels, epochs=5, validation_data=(val_sentences, val_labels), callbacks=[create_tensorboard_callback(SAVE_DIR, 'model_3_GRU')])

Saving TensorBoard log files to: model_logs/model_3_GRU/20220123-132326
Epoch 1/5
215/215 [==============================] - 3s 7ms/step - loss: 0.1511 - accuracy: 0.9378 - val_loss: 0.7182 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0862 - accuracy: 0.9698 - val_loss: 0.8560 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0703 - accuracy: 0.9723 - val_loss: 0.8992 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0586 - accuracy: 0.9747 - val_loss: 1.0886 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0539 - accuracy: 0.9756 - val_loss: 1.1807 - val_accuracy: 0.7703


In [55]:
model_3_preds_probs = model_3.predict(val_sentences)
model_3_preds_probs[:10]

array([[1.6602847e-03],
       [7.1000373e-01],
       [9.9986911e-01],
       [1.0646888e-01],
       [1.5334935e-04],
       [9.9970043e-01],
       [9.0998423e-01],
       [9.9995208e-01],
       [9.9989545e-01],
       [9.9587256e-01]], dtype=float32)

In [56]:
model_3_preds = tf.squeeze(tf.round(model_3_preds_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [57]:
model_3_result = calculate_results(val_labels, model_3_preds)
model_3_result

{'accuracy': 77.03412073490814,
 'f1': 0.7686901866564684,
 'precision': 0.7711671866902318,
 'recall': 0.7703412073490814}

In [58]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

## Model using bidirectional-LSTM RNN 

In [59]:
from tensorflow.keras import layers 
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_4 = tf.keras.Model(inputs, outputs, name='model_4_bidirectional')


In [60]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                             

In [61]:
# Compile
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [62]:
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "bidirectional_RNN")])

Saving TensorBoard log files to: model_logs/bidirectional_RNN/20220123-132335
Epoch 1/5
215/215 [==============================] - 9s 17ms/step - loss: 0.0998 - accuracy: 0.9669 - val_loss: 0.9287 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0498 - accuracy: 0.9777 - val_loss: 1.0825 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0438 - accuracy: 0.9790 - val_loss: 1.1832 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0441 - accuracy: 0.9793 - val_loss: 1.2798 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0397 - accuracy: 0.9800 - val_loss: 1.3485 - val_accuracy: 0.7664


In [63]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[3.5805714e-03],
       [4.5962906e-01],
       [9.9996305e-01],
       [2.2611159e-01],
       [1.4329990e-04],
       [9.9987352e-01],
       [5.5674171e-01],
       [9.9998701e-01],
       [9.9996448e-01],
       [9.9541402e-01]], dtype=float32)

In [64]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [65]:
model_4_results = calculate_results(val_labels, model_4_preds)
model_4_results

{'accuracy': 76.64041994750657,
 'f1': 0.7644156046177798,
 'precision': 0.7676724239336781,
 'recall': 0.7664041994750657}

##  Conv1D model

In [67]:
embedding_test = embedding(text_vectorizer(["This is test sentence"]))
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5,
                        activation='relu',
                        padding='valid')
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [70]:
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64, kernel_size=5, activation='relu', padding='valid', strides=1)(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_5 = tf.keras.Model(inputs, outputs, name='model_5_Conv1D')
model_5.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 conv1d_4 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_4 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 1)              

In [76]:
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 'Conv1d')])

Saving TensorBoard log files to: model_logs/Conv1d/20220123-142422
Epoch 1/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0313 - accuracy: 0.9839 - val_loss: 1.2196 - val_accuracy: 0.7520
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0307 - accuracy: 0.9835 - val_loss: 1.2094 - val_accuracy: 0.7493
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0303 - accuracy: 0.9842 - val_loss: 1.1903 - val_accuracy: 0.7480
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0303 - accuracy: 0.9838 - val_loss: 1.2293 - val_accuracy: 0.7467
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0291 - accuracy: 0.9838 - val_loss: 1.2456 - val_accuracy: 0.7559


In [77]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

array([[1.03327595e-02],
       [9.92193937e-01],
       [9.99917269e-01],
       [7.53839221e-03],
       [1.62993929e-05],
       [9.99051750e-01],
       [4.86853927e-01],
       [9.99923110e-01],
       [9.99996305e-01],
       [9.61790383e-02]], dtype=float32)

In [78]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 0.], dtype=float32)>

In [79]:
model_5_results = calculate_results(val_labels, model_5_preds)
model_5_results

{'accuracy': 75.59055118110236,
 'f1': 0.7536918613918199,
 'precision': 0.7571361526829015,
 'recall': 0.7559055118110236}

In [80]:
# Example of pretrained embedding with universal sentence encoder - https://tfhub.dev/google/universal-sentence-encoder/4
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4") # load Universal Sentence Encoder
embed_samples = embed([sample_sentence,
                      "When you call the universal sentence encoder on a sentence, it turns it into numbers."])

print(embed_samples[0][:50])

tf.Tensor(
[-0.01157032  0.02485909  0.02878048 -0.01271501  0.03971539  0.0882776
  0.02680985  0.05589837 -0.0106873  -0.00597291  0.00639325 -0.0181952
  0.00030816  0.09105889  0.05874643 -0.03180627  0.01512473 -0.05162929
  0.00991365 -0.06865346 -0.04209305  0.02678981  0.03011008  0.00321067
 -0.0033797  -0.04787361  0.02266722 -0.00985925 -0.04063613 -0.0129209
 -0.04666385  0.056303   -0.03949255  0.00517688  0.02495828 -0.07014443
  0.02871508  0.04947681 -0.00633976 -0.08960193  0.02807116 -0.00808363
 -0.01360604  0.0599865  -0.10361787 -0.05195372  0.00232956 -0.02332528
 -0.03758106  0.03327731], shape=(50,), dtype=float32)


In [81]:
# Create a Keras Layer using the USE pretrained layer from tensroflow hub
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name='USE')

In [89]:
model_6 = tf.keras.Sequential([
                               sentence_encoder_layer,
                               layers.Dense(64, activation='relu'),
                               layers.Dense(1, activation='sigmoid')
], name = 'model_6_USE')

model_6.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_7 (Dense)             (None, 64)                32832     
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [90]:
model_6.fit(train_sentences,
            train_labels,
            epochs=5,
            validation_data=(val_sentences, val_labels),
            callbacks=[create_tensorboard_callback(SAVE_DIR, 'model_6_USE')])

Saving TensorBoard log files to: model_logs/model_6_USE/20220123-150308
Epoch 1/5
215/215 [==============================] - 4s 13ms/step - loss: 0.5059 - accuracy: 0.7789 - val_loss: 0.4451 - val_accuracy: 0.7979
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.4141 - accuracy: 0.8180 - val_loss: 0.4358 - val_accuracy: 0.8084
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.4007 - accuracy: 0.8227 - val_loss: 0.4345 - val_accuracy: 0.8097
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3919 - accuracy: 0.8240 - val_loss: 0.4286 - val_accuracy: 0.8110
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3854 - accuracy: 0.8314 - val_loss: 0.4288 - val_accuracy: 0.8150


In [91]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

array([[0.15669984],
       [0.71984494],
       [0.9901584 ],
       [0.20670941],
       [0.7204084 ],
       [0.64652544],
       [0.98397774],
       [0.9774684 ],
       [0.9363867 ],
       [0.09542953]], dtype=float32)

In [92]:
model_6_pred = tf.squeeze(tf.round(model_6_pred_probs))
model_6_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [93]:
model_6_results = calculate_results(val_labels, model_6_pred)
model_6_results

{'accuracy': 81.49606299212599,
 'f1': 0.813013156274745,
 'precision': 0.8186374796992019,
 'recall': 0.8149606299212598}

## Model 7: Same but 10% of data

In [97]:
train_10_percent =  train_df_shuffled[['text', 'target']].sample(frac=0.1, random_state=42)
train_sentences_10_percent = train_10_percent['text'].to_list()
train_labels_10_percent = train_10_percent['target'].to_list()
len(train_sentences_10_percent), len(train_labels_10_percent)

(761, 761)

In [104]:
model_7 = tf.keras.Sequential([
                               sentence_encoder_layer,
                               layers.Dense(64, activation='relu'),
                               layers.Dense(1, activation='sigmoid')
], name = 'model_7_USE_10')

model_7.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
model_7.summary()

Model: "model_7_USE_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_11 (Dense)            (None, 64)                32832     
                                                                 
 dense_12 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [105]:
model_7.fit(train_sentences_10_percent,
            train_labels_10_percent,
            epochs=5,
            validation_data=(val_sentences, val_labels),
            callbacks=[create_tensorboard_callback(SAVE_DIR, 'model_7_USE_10_percent')])

Saving TensorBoard log files to: model_logs/model_7_USE_10_percent/20220123-152127
Epoch 1/5
24/24 [==============================] - 2s 40ms/step - loss: 0.6673 - accuracy: 0.7096 - val_loss: 0.6266 - val_accuracy: 0.8097
Epoch 2/5
24/24 [==============================] - 1s 24ms/step - loss: 0.5947 - accuracy: 0.8003 - val_loss: 0.5482 - val_accuracy: 0.8058
Epoch 3/5
24/24 [==============================] - 1s 25ms/step - loss: 0.5206 - accuracy: 0.8042 - val_loss: 0.4823 - val_accuracy: 0.8136
Epoch 4/5
24/24 [==============================] - 1s 22ms/step - loss: 0.4662 - accuracy: 0.8095 - val_loss: 0.4383 - val_accuracy: 0.8123
Epoch 5/5
24/24 [==============================] - 1s 25ms/step - loss: 0.4290 - accuracy: 0.8160 - val_loss: 0.4083 - val_accuracy: 0.8255


In [106]:
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

array([[0.25137025],
       [0.81350625],
       [0.90206665],
       [0.30993497],
       [0.7903392 ],
       [0.8314074 ],
       [0.89465725],
       [0.93197757],
       [0.82027316],
       [0.0682212 ]], dtype=float32)

In [107]:
model_7_pred = tf.squeeze(tf.round(model_7_pred_probs))
model_7_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [109]:
model_7_result = calculate_results(val_labels, model_7_pred)
model_7_result

{'accuracy': 82.54593175853019,
 'f1': 0.8248354823684468,
 'precision': 0.8257387867002456,
 'recall': 0.8254593175853019}